In [3]:
"""
Analiza una lista de soles candidatos y calcula el porcentaje de datos no nulos
para las columnas científicas clave de ambos datasets.
"""
import pandas as pd

# --- 1. Configuración ---
ARCHIVO_DERIVADO = 'dataset_meda_derived_data.parquet'
ARCHIVO_CALIBRADO = 'dataset_meda_calibrated_data.parquet'

# Lista de los mejores soles que encontramos antes (o cualquier candidato que quieras probar)
SOLES_CANDIDATOS = [2, 1, 3, 0, 4, 5, 6, 7, 8,37, 41, 45, 47, 51, 53, 57, 61, 63, 67, 75]

# Las columnas cuya cobertura queremos verificar
COLUMNAS_CLAVE = [
    'ATS_LOCAL_TEMP1_C',
    'LOCAL_RELATIVE_HUMIDITY',
    'HORIZONTAL_WIND_SPEED',
    'DOWNWARD_LW_IRRADIANCE'
]

# --- 2. Proceso de Diagnóstico ---
print("Iniciando diagnóstico de cobertura de datos por sol y por sensor...")

for sol in SOLES_CANDIDATOS:
    print(f"\n--- Analizando Sol {sol} ---")
    try:
        # Cargamos los datos de ambos archivos para el sol actual
        df_d = pd.read_parquet(ARCHIVO_DERIVADO, filters=[('sol', '==', sol)])
        df_c = pd.read_parquet(ARCHIVO_CALIBRADO, filters=[('sol', '==', sol)])
        
        if df_c.empty or df_d.empty:
            print("  - Faltan datos en uno de los archivos para este sol.")
            continue

        # Hacemos la conversión a Celsius y la fusión
        df_c['ATS_LOCAL_TEMP1_C'] = df_c['ATS_LOCAL_TEMP1'] - 273.15
        df_sol = pd.merge(df_d, df_c, on=['SCLK', 'sol'], how='outer')
        
        total_filas = len(df_sol)
        print(f"  - Total de mediciones (filas) en este sol: {total_filas}")

        # Calculamos el porcentaje de datos completos para cada columna clave
        print("  - Cobertura de datos por sensor:")
        for columna in COLUMNAS_CLAVE:
            if columna in df_sol.columns:
                datos_no_nulos = df_sol[columna].notna().sum()
                porcentaje = (datos_no_nulos / total_filas) * 100
                print(f"    - {columna}: {porcentaje:.1f}% completo")
            else:
                print(f"    - {columna}: No encontrada")

    except Exception as e:
        print(f"  - Ocurrió un error al procesar el sol {sol}: {e}")

print("\n--- Diagnóstico Finalizado ---")

Iniciando diagnóstico de cobertura de datos por sol y por sensor...

--- Analizando Sol 2 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 1 ---
  - Total de mediciones (filas) en este sol: 1780713
  - Cobertura de datos por sensor:
    - ATS_LOCAL_TEMP1_C: 0.1% completo
    - LOCAL_RELATIVE_HUMIDITY: 55.8% completo
    - HORIZONTAL_WIND_SPEED: 99.9% completo
    - DOWNWARD_LW_IRRADIANCE: 72.1% completo

--- Analizando Sol 3 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 0 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 4 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 5 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 6 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 7 ---
  - Faltan datos en uno de los archivos para este sol.

--- Analizando Sol 8 ---
  - Faltan datos en uno de los archivos para este s